In [87]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Initialize faker
fake = Faker()

class AESCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX)
        ciphertext, tag = cipher.encrypt_and_digest(data)
        return cipher.nonce, b64encode(ciphertext).decode('utf-8')

# Define a function to generate a customer
def generate_customer_profile():
    profile = fake.profile()
    return {
        "name": profile['name'],
        "address": fake.address().replace('\n', ', '),
        "email": profile['mail'],
        "phone_number": fake.phone_number(),
        "social_security_number": fake.ssn(),
        "job": profile['job'],
        "company": profile['company'],
        "residence": fake.address().replace('\n', ', '),
        "current_location": fake.address().replace('\n', ', '),
        "website": profile['website'][0] if profile['website'] else None,
        "username": profile['username'],
        "age": fake.random_int(min=18, max=80, step=1),
        "gender" : profile['sex'],
        "Grades": fake.random_int(min=0, max=100, step=1),
        "preferred_language": fake.language_code(),
        "preferred_currency": fake.currency_code(),
        "job_title": profile['job'],
        "national_id": fake.random_int(min=1000000000, max=9999999999, step=1),
        "height": fake.random_int(min=100, max=200, step=1),
        "weight": fake.random_int(min=100, max=200, step=1),
    }

# Define a function to generate a transaction
def generate_transaction(name):
    return {
        "name": name,
        "amount": f"${fake.random_int(min=1, max=1000, step=1):.2f}",
        "date": fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
    }

# Generate customers
customers = [generate_customer_profile() for _ in range(10)]
customers_df = pd.DataFrame(customers)



In [88]:

# Generate transactions for each customer randomly picking a customer by name
transactions = [generate_transaction(random.choice(customers)["name"]) for _ in range(20)]
transactions_df = pd.DataFrame(transactions)
transactions_df.head()

,name,amount,date
0,Michelle Barry,$739.00,2022-08-12 06:14:43
1,Jason Farmer,$702.00,2023-04-16 13:34:40
2,Troy White,$166.00,2022-10-19 22:57:24
3,Jason Farmer,$90.00,2023-01-04 17:52:36
4,Melissa Huang,$81.00,2023-01-11 09:45:20


In [89]:

# Encrypt the data in the CRM
password = get_random_bytes(16)  # In a real system, never hard-code the password
cipher = AESCipher(password)


In [90]:

encrypted_df = pd.DataFrame()
encryption_key = pd.DataFrame()
for column in customers_df.columns:
    encrypted_column = []
    key_column = []
    for data in customers_df[column]:
        iv, ct = cipher.encrypt(str(data).encode('utf-8'))
        encrypted_column.append(ct)
        key_column.append(iv)
    encrypted_df[column] = encrypted_column
    encryption_key[column] = key_column

print("password: ", password)
# save the encrypted data (first five rows as a markdown table)
md_table = encrypted_df.head().to_markdown()
# save md table to file
with open("encrypted_data.md", "w") as f:
    f.write(md_table)

# Read the README.md file
readme_text = ""
with open("README.md", "r") as f:
    readme_text = f.read()
# open the README.md file and add the encrypted data at the end
with open("README.md", "w") as f:
    # read what is currently in the README.md file
    readme = readme_text
    # add the original dataframe (fake data)
    readme += "\n\n## Original Data generated with Faker\n\n"
    readme += customers_df.head().to_markdown()
    # add the encrypted data
    readme += "\n\n## Encrypted Data\n\n"
    readme += md_table
    # write the new README.md file
    f.write(readme)

# Print the encrypted data
encrypted_df.head()

password:  b'\x1c\x10\x93\xf1\xc8\xbe7\x07\xa9%\xf6\n\x7f\xcfR['


,name,address,email,phone_number,social_security_number,job,company,residence,current_location,website,username,age,gender,Grades,preferred_language,preferred_currency,job_title,national_id,height,weight
0,me+7FEI+LQXdh2f+y5g=,zXtBO512N+4NIlYk7AkY11wZoEty8DRJqEr4LgPSq2Sd0I...,nKc+GLIkCfe9RAxL5BRkFCBo6xDiusMbWv+bfA==,nmQZx3L2MaA4THv3tx47yGA4,5J6uZgS4BY83tLM=,hUWkOgfdIdhx0pL7QQpEZOLQFXmBM+R1,UMs+kaW3esigtuZyoTuGKAaF/YqMNWC8l+M=,4Y0SjnqQo6yuW1pzaRqyutYCPA6nX5qD0DdPvHU1HqkckL...,yaih6zTtCAqzvMDmhHeCaHs6lLJx2FDdEgs7Qph31WjIF4...,LPnMUnuWGqM5j80iUbInSV8IGMd6/O8=,RFNk/jUi2hebLRE=,M9c=,6A==,RXk=,YRIT,P5LJ,IXCHFowT7a/3wLCLYMDMbaYbalU2k64B,Hf+6rkhQKGQ+Tw==,namU,w+kt
1,D5km1jrziWgH2usUYw==,eJQ+hUQPfh+PnspwR7uIEKXShPf8B+2Rooi+rETsAw6Nue...,2mE7P8z3g/TBK/H1FXIgt5w=,oj0Ni0zU3yfcKaKwoA==,r91w4zyEw2Xi0PQ=,v3cLz1O3q2VMwbzkXQ1K5HRYcY3ghw==,7AyXxWLl8Cfh8kCY0G4ycucF+g+G0gq6a+k=,wM4yEKC85tOpWfdcxgxhz71zO23MiQ0QXK+KWlwB97sU5z...,Jyb/t/yv3FV25AYK7RDL7joodjmBPQ/kPIDZ+2EceRHJ4Y...,zuO/wemUZLho2xsyfx4znBkZFtEk,+hvYyOU1+Zw=,4LA=,lA==,sU4=,eWU=,MpOc,Nwiq/kNhDpQeRMTaoppblmIlkN/pGQ==,C7kC/Q47CZWIWQ==,gltA,FIpr
2,lQyQaOZvMDj54Q==,LXVTm4yNGzNFHAxZoKvpT6viT7jTyEnI4GRrVldoZrtEH/...,rxkuNgyFltMuF0/P3JVgG0kiDNg=,TjEeUxTHpJnX2/Oo4n/JCVrp,Te/5gvfXhMK9J8c=,jFonaS2f/1ABWSRY3REBhqXw3bI=,c0tqedRRT0AMyz9fuZ2Y1l+FQF3aYghXJhI=,6QOUNNSNpoNTLl3WJDhTsL5sRpvFK8jcF88cTpdv7miJON...,qJ+mWHQ02PNCwz+aJn7yGVr/SkfKiwLGJ6kXv0KR474=,tE/9/3cmP6zJmDThldJD8qxZ+225,NooYYxe/7pA=,qFw=,4w==,5g==,J4o=,Isdd,UOxDgMvlx8uTibxuEWE3DSQuOPQ=,RwzbtwDUg0YPIQ==,aacW,Zzc+
3,i1ytH5EU78xv3dNpHao=,ojmkOh58vh7EwZ+CLl/nSTR4pNiUW3pzdJzP4ibeGPQ=,X5G7LDpY9RnqTxlRgKtw/ITjOg==,AkjnmQftGm2+NsMv7g9Y,ROePQ4xSHZw2gxw=,YRWILB9tEaCwYJfziAJOGKQcCg==,ImIwVaeDdQBZgGZODGA=,peVnSm9HEIDEU3SIGpzFPfUqLirO99EkF121N2iuxvY=,Q+YnGleGESy/2wnbWcHFVCqDXhMZkKqlQE2zMC3XA8vLe2...,jZWpYpMTfXHH7YuUQzro58ssP52G99VA+32m9Q==,O90gAPOgvlDG1jQs,SsU=,MQ==,w8g=,RiA=,3hfI,bKZjc1CFQg/Ar3c/9GectiKhmw==,eLnSs+1tefuzqw==,uCdY,cdpw
4,C0k0KnE9vPao1IKZd80iYt08,2P46ZDIUQYS7QBvFAmtB8iZrN/tLH66wTM8AgeYc1Xadkp...,oUspn7DNBo2KRvm4N4yjo45jAWybHs7La2C9,H3fdpD1U6Hseoc3FXkMdC6Q=,mDES8kB3xO0MDAM=,snEiGUgOSPk3jeWoz75rI2y6bkJxWA==,VKuw1MWBzI44,dIzHoDcHx0eMw8Hcqf5A8pRa/WFDrP3kdX2rfBEGu/4DWl...,6oBrdODCvSdX97CmwcnvuLJTHsVfGNvqHXhOZLk1L2s=,qwZI/B1fVti8n5yjlwpPu53TbFbR,rhLcpP87SjJ2bc7q,gkI=,8A==,fv8=,VVYP,1gpD,Qz6eBtNuP8FJcR5SGNFDUiEXaJHH3w==,JXA2l34ghAlLtg==,DPci,Igek
